#  Medical Symptom Analysis using Apriori Algorithm

This notebook presents a data-driven medical assistant based on association rule mining. The goal is to identify potential co-occurring conditions based on patient symptoms using the **Apriori algorithm**.

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv('disease_transactions_1500.csv')

In [3]:
transactions = df.values.tolist()
transactions = [[item for item in row if pd.notna(item)] for row in transactions]

In [4]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [5]:
frequent_itemsets = apriori(df_encoded, min_support=0.02, use_colnames=True)

In [6]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

In [7]:
print("Frequent itemsets:")
frequent_itemsets

Frequent itemsets:


,support,itemsets
0,0.162667,(Arthritis)
1,0.123333,(Asthma)
2,0.201333,(Chronic Kidney Disease)
3,0.178000,(Coronary Heart Disease)
4,0.115333,(Depression)
5,0.185333,(Diabetes)
6,0.299333,(Hyperlipidemia)
7,0.310000,(Hypertension)
8,0.184000,(Obesity)
9,0.212000,(Stroke)


In [8]:
print("Association rules:")
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

Association rules:


,antecedents,consequents,support,confidence,lift
0,(Depression),(Arthritis),0.042000,0.364162,2.238700
1,(Arthritis),(Depression),0.042000,0.258197,2.238700
2,(Chronic Kidney Disease),(Diabetes),0.053333,0.264901,1.429320
3,(Diabetes),(Chronic Kidney Disease),0.053333,0.287770,1.429320
4,(Chronic Kidney Disease),(Hyperlipidemia),0.040667,0.201987,0.674789
...,...,...,...,...,...
103,"(Obesity, Diabetes)","(Hypertension, Hyperlipidemia)",0.064667,0.505208,4.163805
104,(Hypertension),"(Hyperlipidemia, Obesity, Diabetes)",0.064667,0.208602,2.744765
105,(Hyperlipidemia),"(Hypertension, Obesity, Diabetes)",0.064667,0.216036,2.064035
106,(Obesity),"(Hypertension, Hyperlipidemia, Diabetes)",0.064667,0.351449,3.847985


##  Medical Assistant Chatbot (Apriori-Based)

This chatbot suggests possible conditions you might experience based on your selected symptom, using association rules derived from the Apriori algorithm. Please select a symptom from the dropdown below to get suggestions.


In [9]:
import pandas as pd
from ipywidgets import Dropdown, Button, Output, VBox, Label
from IPython.display import display

In [10]:
# Extract all unique symptoms from the antecedents (left side of rules)
all_symptoms = set()
for s in rules['antecedents']:
    if isinstance(s, frozenset):
        all_symptoms.update(s)
    elif isinstance(s, str):
        all_symptoms.update([s])
    else:
        all_symptoms.update(list(s))

# Convert to sorted list for dropdown
symptom_list = sorted(list(all_symptoms))


In [11]:
# Create dropdown widget
dropdown = Dropdown(
    options=symptom_list,
    description='Select symptom:',
    style={'description_width': 'initial'}
)

# Create output area
output = Output()


In [12]:
def medical_assistant(selected_symptom):
    output.clear_output()
    
    
    matched = rules[rules['antecedents'].apply(lambda x: selected_symptom in list(x))]

    with output:
        if matched.empty:
            print(f"No strong associations found for: {selected_symptom}")
        else:
            result_dict = {}
            for _, row in matched.iterrows():
                for consequent in row['consequents']:
                    if consequent not in result_dict or row['confidence'] > result_dict[consequent]:
                        result_dict[consequent] = row['confidence']

            
            print(f"🤖 Based on having '{selected_symptom}', you might also experience:")
            for disease, confidence in sorted(result_dict.items(), key=lambda x: x[1], reverse=True):
                print(f"- {disease} (confidence: {confidence:.2f})")
            
            print("\n📌 Please consult a doctor for medical advice.")



In [ ]:
# Create button and link it to the function
button = Button(description="Get Suggestion", button_style='success')

# When clicked, call the medical_assistant function
def on_click_handler(b):
    medical_assistant(dropdown.value)

button.on_click(on_click_handler)

# Display everything
display(VBox([
    Label("🩺 Select a symptom to see possible related conditions:"),
    dropdown,
    button,
    output
]))

##  Save Apriori Rules to a Pickle File

To avoid re-running the Apriori algorithm every time, we can save the generated association rules into a `.pkl` file. This file can later be loaded directly in a Flask app for fast and efficient deployment.


In [14]:
import pickle

with open("apriori_rules.pkl", "wb") as f:
    pickle.dump(rules, f)

print(" Rules saved successfully to apriori_rules.pkl")


 Rules saved successfully to apriori_rules.pkl
